# EE954 : Deep Learning Fundamentals
### Assignment 1

**Group Number:** 12  
**Team Members:**  
- Lokesh  (Roll No: 241562482)  
- Akshay (Roll No: _____)

---



#### Importing Libraries &#8595;

In [3]:
!pip install kagglehub
import numpy as np
from torchvision import datasets
import kagglehub
import shutil
import os
import pandas as pd
from sklearn.model_selection import train_test_split

#### 1.1 Download and Split &#8595;

General Instructions 
• Late submissions will not be accepted. 
• Any form of plagiarism will result inpenalties. If you refer to any online material or books,cite them properly. 
• You may use Google Colab or Kaggle to train your models. 
• The use of the Numpy library is permitted. 
• The use of Tensor Flow library is strictly prohibited. 
• The use of PyTorch library is allowed with restrictions.

DatasetPreparation (5Marks)

1.1 Download and Split (2Marks)

•Download the Fashion-MNIST dataset. You can either download it from here,or import it directly into your code using PyTorch’s torchvision.datasets module. 

• Both sources provide separate training and test splits; however, you will need to create a separate validation set from the training data.

From Perplexity.AI - '

Fashion-MNIST is a widely used machine learning dataset consisting of 70,000 grayscale images (28x28 pixels) of fashion items from 10 categories, such as T-shirts, trousers, dresses, and shoes. There are 60,000 images for training and 10,000 for testing. Each image is labeled with one of the 10 clothing classes. Fashion-MNIST was designed as a more challenging, modern replacement for the original MNIST handwritten digits dataset, while maintaining the same format and structure for easy benchmarking and comparison of machine learning algorithms.

Each example is a 28x28 grayscale image of a fashion item, labeled with one of 10 classes: 0: T-shirt/top 1: Trouser 2: Pullover 3: Dress 4: Coat 5: Sandal 6: Shirt 7: Sneaker 8: Bag 9: Ankle boot

'

Documentation on dataset - https://github.com/zalandoresearch/fashion-mnist

In [4]:
# where the MNIST data will be saved permanently
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "..", "datasets")

# Download dataset to the default directory
path = kagglehub.dataset_download("zalando-research/fashionmnist")

# Copy the downloaded the dataset to the desired path
shutil.copytree(path, DATA_DIR, dirs_exist_ok=True)

if os.path.exists(DATA_DIR) and os.listdir(DATA_DIR):
    print(f"Dataset copied successfully to: {DATA_DIR}")
    print("Contents:")
    for f in os.listdir(DATA_DIR):
        print(" -", f)
else:
    print("Dataset folder is missing or empty!")

train_file = os.path.join(DATA_DIR, "fashion-mnist_train.csv")
test_file = os.path.join(DATA_DIR, "fashion-mnist_test.csv")

df = pd.read_csv(train_file)

X = df.drop('label', axis=1)
y = df['label']

X = X.to_numpy()
y = y.to_numpy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=42)

100%|██████████| 68.8M/68.8M [00:17<00:00, 4.18MB/s]

Extracting files...


Dataset copied successfully to: c:\Users\aksha\OneDrive\Desktop\Education Courses\IITK AI ML\GitHub\EE954_DLF\..\datasets
Contents:
 - fashion-mnist_test.csv
 - fashion-mnist_train.csv
 - t10k-images-idx3-ubyte
 - t10k-labels-idx1-ubyte
 - train-images-idx3-ubyte
 - train-labels-idx1-ubyte


In [6]:
X_train.shape

(48000, 784)

### The dataset downloaded from Kaggle is already flattened. 
Notes from Kaggle:  

'Content
Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. The training and test data sets have 785 columns. The first column consists of the class labels (see above), and represents the article of clothing. The rest of the columns contain the pixel-values of the associated image.

To locate a pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27. The pixel is located on row i and column j of a 28 x 28 matrix.
For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.'

#### 1.2 Data Preprocessing (3 Marks)
• Define a preprocess function to preprocess the images. (The function should have
the same name).

• The function should flatten the images.

• The function should also normalize the pixel values to the range [0, 1]. Depending
on how you have implemented the code so far, the pixel values might already be
normalized to this range. However, for clarity and completeness, include an explicit
normalization step regardless. &#8595;

In [10]:
print(f"X Shape: {X.ndim}")
print(f"y Shape: {y.ndim}") 
print(f"Max pixel value is {X.max()}")
print(f"Min pixel value is {X.min()}")  

X Shape: 2
y Shape: 1
Max pixel value is 255
Min pixel value is 0


In [14]:
def preprocess_(X):
    if X.ndim == 3:                     # We know that the image is already flattened when used from the KaggleHub dataset, however we are checking for safety
        X = X.reshape(X.shape[0], -1)   # Flatten each image
    if X.max() > 1:
        X = X / 255.0                  # We also saw from above that the dataset while being flattened is not normalized - so we normalize it here
    return X


class PreprocessedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
  
    def preprocess(self):
        return preprocess_(self.dataset)  # Apply preprocessing


# Wrap your existing datasets
X_train = PreprocessedDataset(X_train)
X_val = PreprocessedDataset(X_val)

NameError: name 'Dataset' is not defined

@Lokesh - please see the suggested code below to changes in class

class PreprocessedDataset:
    def __init__(self, X):
        self.raw_data = X  # Store original 2D array (n_samples, 784)
        self.processed_data = None
    
    def preprocess(self):
        """Applies preprocessing and stores result"""
        self.processed_data = preprocess_(self.raw_data)
        return self.processed_data  # Returns processed array
    
    # Optional: Add array-like access if needed
    def __getitem__(self, idx):
        if self.processed_data is None:
            raise ValueError("Call preprocess() first")
        return self.processed_data[idx]
    
    def __len__(self):
        return len(self.raw_data)


# Assuming X_train is your 2D NumPy array (n_samples, 784)
train_dataset = PreprocessedDataset(X_train)

# Apply preprocessing (stores result internally)
processed_array = train_dataset.preprocess() 

# Access individual samples (optional)
sample = train_dataset[0]  # Only works after calling preprocess()




In [11]:
###   Checks again

print(f"X Shape of Training Dataset: {X_train.ndim}")
print(f"X Shape of Validation Dataset: {X_val.ndim}") 
print(f"Max pixel value in Training Dataset is {X_train.max()}")
print(f"Min pixel value in Training Dataset is {X_train.min()}")  
print(f"Max pixel value in Validation Dataset is {X_val.max()}")
print(f"Min pixel value in Validation Dataset is {X_val.min()}")  

X Shape of Training Dataset: 2
X Shape of Validation Dataset: 2
Max pixel value in Training Dataset is 255
Min pixel value in Training Dataset is 0
Max pixel value in Validation Dataset is 255
Min pixel value in Validation Dataset is 0


 #### Activation Function &#8595;

In [6]:
def activation(activation_type, z):
    if activation_type == 'relu':
        return np.maximum(0, z)
    if activation_type == 'sigmoid':
        return 1/(1+np.exp(-z))
    if activation_type == 'tanh':
        return (np.tanh(z))
    else:
        raise ValueError("Unsupported activation type. Use 'relu', 'sigmoid', or 'tanh'.")
    

#### Forward Propagation  &#8595;

In [5]:
X = df.drop('label', axis=1)
y = df['label']

#Total number of classes to be predicted
n_classes = len(np.unique(y))
print(n_classes)

print(f"X dataframe shape{X.shape}")
print(f"y dataframe shape{y.shape}")

10
X dataframe shape(60000, 784)
y dataframe shape(60000,)


#### Backward Propagation  &#8595;

In [ ]:
class MLP(self, PreprocessedDataset):
    def init __super():
        self.PreprocessedDataset
    init __self__():
   
    X_train = preprocess(X_train)
    X_val = preprocess(X_val)

nn = MLP()
for _ in range(epochs):
    
    nn.forwardpropagation()
    nn.backpropagation()
    